In [10]:
# !wget https://fever.ai/download/fever/train.jsonl
!wget https://fever.ai/download/fever/shared_task_dev.jsonl
!wget https://fever.ai/download/fever/wiki-pages.zip

--2024-04-03 16:22:10--  https://fever.ai/download/fever/shared_task_dev.jsonl
Resolving fever.ai (fever.ai)... 3.161.150.40, 3.161.150.98, 3.161.150.110, ...
Connecting to fever.ai (fever.ai)|3.161.150.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4349935 (4.1M) [binary/octet-stream]
Saving to: 'shared_task_dev.jsonl.1'

shared_task_dev.jso 100%[===================>]   4.15M  7.54MB/s    in 0.6s    

2024-04-03 16:22:12 (7.54 MB/s) - 'shared_task_dev.jsonl.1' saved [4349935/4349935]

--2024-04-03 16:22:13--  https://fever.ai/download/fever/wiki-pages.zip
Resolving fever.ai (fever.ai)... 3.161.150.40, 3.161.150.98, 3.161.150.110, ...
Connecting to fever.ai (fever.ai)|3.161.150.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1713485474 (1.6G) [application/zip]
Saving to: 'wiki-pages.zip'

wiki-pages.zip      100%[===================>]   1.60G  47.4MB/s    in 34s     

2024-04-03 16:22:48 (47.4 MB/s) - 'wiki-pages.zip' saved [17

In [ ]:
!unzip -o wiki-pages.zip
!rm wiki-pages.zip

In [1]:
!git clone https://github.com/Prayas-Agrawal/728-2

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into '728-2'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (3/3), 87.80 MiB | 10.55 MiB/s, done.


In [9]:
!unzip ./728-2/finetune_data.zip

Archive:  ./728-2/finetune_data.zip
replace finetune_data? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [10]:
# !rm -rf  ./728-2

In [1]:
!pip install faiss-cpu
!pip install sentence_transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 54.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
import numpy as np
import os
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, models, InputExample, datasets, losses
import faiss
import itertools
import torch
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score



In [ ]:
# !pip install langchain
# !pip install --upgrade --quiet langchain-text-splitters tiktoken

# !pip install huggingface_hub

In [3]:
train_df = pd.read_json("train.jsonl", lines=True)
# wiki_df = pd.read_json("train.jsonl", lines=True)
# dev_df = pd.read_json("shared_task_dev.jsonl", lines=True)

In [4]:
train_df.head()

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"


In [ ]:
# def make_train_df(train_df, wiki_df):
#   samples = {"claim": [], "text": []}
#   for i,r in tqdm( train_df.iterrows(), total=train_df.shape[0]):
#     ev = r["evidence"]
#     claim = r["claim"]
#     ids = []
#     for ann_data in ev:
#       for data in ann_data:
#         id = data[2]
#         if id != None and id not in ids:
#           if(id not in wiki_df.index):
#             continue
#           text = get_text_by_id(wiki_df, id)
#           ids.append(id)
#           samples["claim"].append(claim)
#           samples["text"].append(text)
#           # samples.append(InputExample(texts=[claim, text]))
#   samples = pd.DataFrame(samples)
#   return samples

In [5]:
train_df.iloc[0]["evidence"]

[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [22]:
wiki_df.head()

,text,lines
id,,
Gergely_Sárközy,Gergely Sárközy is a Hungarian musician who pl...,0\tGergely Sárközy is a Hungarian musician who...
Go_Osaka,"is a Japanese writer of crime fiction , hardbo...","0\tis a Japanese writer of crime fiction , har..."
George_Hodson_-LRB-disambiguation-RRB-,George Hodson was a baseball pitcher . Georg...,0\tGeorge Hodson was a baseball pitcher .\tGeo...
Gnarp,Gnarp is a locality situated in Nordanstig Mun...,0\tGnarp is a locality situated in Nordanstig ...
"George_Douglas,_Master_of_Angus","George Douglas , Master of Angus -LRB- 1469 --...","0\tGeorge Douglas , Master of Angus -LRB- 1469..."


In [5]:
# model_checkpoint = "distilbert-base-uncased"
# model = AutoModel.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# word_embedding_model = models.Transformer('distilroberta-base')

# word_embedding_model = models.Transformer('distilbert-base-uncased')
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
emb_dim = 384
ret_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# test = tokenizer.encode("some sentence", truncation=True, max_length=512)
# test2 = tokenizer.batch_encode_plus(["another sentence", "another sentence"], return_tensors="pt", truncation=True, max_length=512)
# print(test2)
# ret = model(test2.input_ids).last_hidden_state

# print(ret[:, 0, :].shape)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def mySplit(iterable, n=1):
    l = len(iterable)
    ret = []
    for ndx in range(0, l, n):
        ret.append(iterable[ndx:min(ndx + n, l)])
    return ret

def get_sentence_embed(model, text):
  # text = mySplit(text, 16)
  print("encoding num sent: ", len(text))
  emb = model.encode(text,show_progress_bar=True)
  print("embed sent shape: ", emb.shape)
  return emb

def get_sentence_embed__OLD__(text):
  text = mySplit(text, 16)
  print("text", len(text))
  emb = []
  for i, batch in enumerate(text):
    print("doing batch", i, len(batch))
    tokens = tokenizer.batch_encode_plus(batch, return_tensors="pt", truncation=True, max_length=512, padding=True)
    emb_batch = model(tokens.input_ids).last_hidden_state[:, 0, :]
    # emb.append(emb_batch)
    print("emb shape", emb_batch.shape)
    emb = emb + emb_batch.tolist()
  emb = np.asarray(emb, dtype=np.int32)
  print("embed", emb.shape)
  return emb

In [12]:
from pathlib import Path
import os.path

def load_wiki_df():
    dir = "wiki-pages"
    wiki_df = []
    for f in os.listdir(dir):
        if(f.endswith("jsonl")):
            print("loading file", f)
            wiki_df.append(pd.read_json(os.path.join(dir, f), lines=True))
    wiki_df = pd.concat(wiki_df, ignore_index=True)
    wiki_df.set_index('id', inplace=True)
    return wiki_df

def save_ids():
  dir = "wiki-pages"
  wiki_df = []
  saveDir = "./ids/"
  Path(saveDir).mkdir(parents=True, exist_ok=True)
  for f in os.listdir(dir):
    if(f.endswith("jsonl")):
      print("saving ids of file", f)
      df = pd.read_json(os.path.join(dir, f), lines=True)
      ids = df["id"]
      fPath = saveDir + f + ".csv"
      ids.to_csv(fPath, index=False)

def encode_wiki(model, prefix):
    dir = "wiki-pages"
    wiki_df = []
    emb = []
    saveDir = "./encoded_corpus/" + prefix + '/'
    Path(saveDir).mkdir(parents=True, exist_ok=True)
    for f in os.listdir(dir):
        if(f.endswith("jsonl")):
            fname = f + '.npy'
            fPath = saveDir + fname
            if os.path.isfile(fPath):
                print("already encoded", f)
                continue
            print("encoding file", f)
            df = pd.read_json(os.path.join(dir, f), lines=True)
            text = df["text"].tolist()
            emb = get_sentence_embed(model, text)
            with open(fPath, 'wb') as f:
                np.save(fPath, emb)
    print("=========DONE ENCODING=====")

      # index.add_with_ids(encoded_data, np.arange(len(data)))

wiki_df = load_wiki_df()
# save_ids()
# encode_wiki(model, "multi-qa-MiniLM-L6-cos-v1")

loading file wiki-047.jsonl
loading file wiki-003.jsonl
loading file wiki-036.jsonl
loading file wiki-034.jsonl
loading file wiki-108.jsonl
loading file wiki-012.jsonl
loading file wiki-070.jsonl
loading file wiki-028.jsonl
loading file wiki-020.jsonl
loading file wiki-008.jsonl
loading file wiki-072.jsonl
loading file wiki-062.jsonl
loading file wiki-019.jsonl
loading file wiki-018.jsonl
loading file wiki-001.jsonl
loading file wiki-094.jsonl
loading file wiki-086.jsonl
loading file wiki-033.jsonl
loading file wiki-059.jsonl
loading file wiki-015.jsonl
loading file wiki-104.jsonl
loading file wiki-082.jsonl
loading file wiki-025.jsonl
loading file wiki-084.jsonl
loading file wiki-005.jsonl
loading file wiki-078.jsonl
loading file wiki-058.jsonl
loading file wiki-091.jsonl
loading file wiki-009.jsonl
loading file wiki-031.jsonl
loading file wiki-037.jsonl
loading file wiki-029.jsonl
loading file wiki-060.jsonl
loading file wiki-052.jsonl
loading file wiki-097.jsonl
loading file wiki-02

In [9]:
# from IPython.display import FileLink 
# FileLink('ids.zip')


/kaggle/working/ids.zip

In [ ]:
def get_text_by_id(df, id):
  return df.loc[id]["text"]

def get_text_by_id__LAZY__(id):
  dir = "wiki-pages"
  for f in os.listdir(dir):
    if(f.endswith("jsonl")):
      df = pd.read_json(os.path.join(dir, f), lines=True)
      _text = df.loc[df["id"] == id]["text"].values
      if _text.__len__() == 0:
        continue
      _text = _text[0]
      return _text
  print("failed to find", id)
  return ""

def make_ft_df__NEGATIVE_SAMPLED__(train_df, wiki_df, neg_sample = 10):
  ft_df = train_df.copy()
  ft_df["doc_ids"] = [[] for i in range(ft_df.shape[0])]
  ft_df["doc_text"] = [[] for i in range(ft_df.shape[0])]
  ft_df["neg_ids"] = [[] for i in range(ft_df.shape[0])]
  ft_df["neg_text"] = [[] for i in range(ft_df.shape[0])]

  for i,r in ft_df.head().iterrows():
    ev = r["evidence"]
    ids = []
    texts = []
    for ann_data in ev:
      for data in ann_data:
        id = data[2]
        if id != None:
          ids.append(id)
          texts.append(get_text_by_id(wiki_df, id))
    neg = wiki_df[~wiki_df["id"].isin(ids)].loc[:, "id"].sample(n=neg_sample).tolist()
    neg_texts = [get_text_by_id(wiki_df, neg_id) for neg_id in neg]
    ft_df.at[i,"doc_ids"] = ids
    ft_df.at[i,"doc_text"] = texts
    ft_df.at[i,"neg_ids"] = neg
    ft_df.at[i,"neg_text"] = neg_texts
  return ft_df

def make_ft_df(train_df, wiki_df):
  samples = {"claim": [], "text": []}
  for i,r in tqdm( train_df.iterrows(), total=train_df.shape[0]):
    ev = r["evidence"]
    claim = r["claim"]
    ids = []
    for ann_data in ev:
      for data in ann_data:
        id = data[2]
        if id != None and id not in ids:
          if(id not in wiki_df.index):
            continue
          text = get_text_by_id(wiki_df, id)
          ids.append(id)
          samples["claim"].append(claim)
          samples["text"].append(text)
          # samples.append(InputExample(texts=[claim, text]))
  samples = pd.DataFrame(samples)
  return samples


ft_df = make_ft_df(train_df, wiki_df)


In [ ]:
samples = []
for i, row in ft_df.iterrows():
  samples.append(InputExample(texts=[row["claim"], row["text"]]))

In [ ]:
loader = datasets.NoDuplicatesDataLoader(samples, batch_size = 32 )

In [ ]:
word_embedding_model = models.Transformer('distilbert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
loss = losses.MultipleNegativesRankingLoss(model)
epochs = 2
warmup = int(0.1*len(loader))
model.fit(train_objectives=[(loader, loss)], epochs = epochs, warmup_steps = warmup, output_path = "pa2_model", show_progress_bar = True, )

# FAISS

In [10]:
def index_corpus(index, emb_path):
    emb = []
    ids = []
    num2Id = {}
    currHead = 0
    for f in os.listdir(emb_path):
        if(f.endswith(".npy")):
            idFile = "./ids/" + f[0:-4] + ".csv"
            print("indexing file", f, idFile )
            encoded_data = np.load(os.path.join(emb_path, f))
            _ids = pd.read_csv(idFile)
            length = _ids.shape[0]
            d = {i: _ids.iloc[i-currHead].values[0]  for i in range(currHead, currHead + length)}
            num2Id.update(d)
            index.add_with_ids(encoded_data, np.arange(currHead, currHead + length))
            currHead += length
    print("=========DONE INDEXING=====")
    return num2Id
    
index = faiss.IndexIDMap(faiss.IndexFlatIP(emb_dim))
num2Id = index_corpus(index, "/kaggle/input/wiki-encoded-multi-qa-minilm-l6-cos-v1/encoded_corpus/multi-qa-MiniLM-L6-cos-v1" )

    

indexing file wiki-007.jsonl.npy ./ids/wiki-007.jsonl.csv
indexing file wiki-072.jsonl.npy ./ids/wiki-072.jsonl.csv
indexing file wiki-095.jsonl.npy ./ids/wiki-095.jsonl.csv
indexing file wiki-032.jsonl.npy ./ids/wiki-032.jsonl.csv
indexing file wiki-051.jsonl.npy ./ids/wiki-051.jsonl.csv
indexing file wiki-059.jsonl.npy ./ids/wiki-059.jsonl.csv
indexing file wiki-102.jsonl.npy ./ids/wiki-102.jsonl.csv
indexing file wiki-092.jsonl.npy ./ids/wiki-092.jsonl.csv
indexing file wiki-052.jsonl.npy ./ids/wiki-052.jsonl.csv
indexing file wiki-003.jsonl.npy ./ids/wiki-003.jsonl.csv
indexing file wiki-067.jsonl.npy ./ids/wiki-067.jsonl.csv
indexing file wiki-016.jsonl.npy ./ids/wiki-016.jsonl.csv
indexing file wiki-024.jsonl.npy ./ids/wiki-024.jsonl.csv
indexing file wiki-033.jsonl.npy ./ids/wiki-033.jsonl.csv
indexing file wiki-040.jsonl.npy ./ids/wiki-040.jsonl.csv
indexing file wiki-011.jsonl.npy ./ids/wiki-011.jsonl.csv
indexing file wiki-096.jsonl.npy ./ids/wiki-096.jsonl.csv
indexing file 

In [11]:
def search(encoder, index, query, num2Id, wiki_df, k=1):
    query_vector = encoder.encode([query])
    top_k = index.search(query_vector, k)
    docs = [wiki_df.loc[num2Id[_id]].values[0] for _id in top_k[1][0]]
    lines = [wiki_df.loc[num2Id[_id]].values[0] for _id in top_k[1][0]]
    ids = [num2Id[_id] for _id in top_k[1][0]]
    ids = [num2Id[_id] for _id in top_k[1][0]]
    return ids, docs,lines

search(ret_model, index, "How old are you?", num2Id, wiki_df, k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(['How_Old_Are_You?_-LRB-song-RRB-',
  'Young_adult_-LRB-psychology-RRB-',
  'How_Old_Are_You?_-LRB-album-RRB-',
  'How_Old_Are_You?_-LRB-film-RRB-',
  'At_My_Age'],
 ["`` How Old Are You '' is a single released by Robin Gibb , the third and final single released from the album of the same name in October 1983 . It reached # 93 in the UK and # 37 in Germany . `` How Old Are You '' likewise has the rhythm guitar beat , bang-away drumming , a powerful horn and synthesizer blend . ",
  "A young adult is generally a person in the age range of 20 to 39 -LRB- or 40 -RRB- , although definitions and opinions , such as Erik Erikson 's stages of human development , vary . The young adult stage in human development precedes middle adulthood . A person in the middle adulthood stage ages from 40 -LRB- or 41 -RRB- to 64 . In old age , a person is 65 years old or older . ",
  "How Old Are You ? is the second solo album released by British rock musician Robin Gibb it released in 1983 , thirteen years 

In [ ]:
def recall(preds, ground, k=2):
    return set.intersection(set(preds[:k]), set(ground))/len(ground)

def eval_retriever(ret_model, index, num2Id, wiki_df, datasetPath, k=2):
    df = datasetPath
    for i, r in df.iterrows():
        preds = search(ret_model, index, r["query"], num2Id, wiki_df, 5)
        

# NLI Model

In [18]:
label2Num = {
    "SUPPORTS": 0,
    "REFUTES": 1,
    "NOT ENOUGH INFO": 2
}

num2Label = {
    0: "SUPPORTS",
    1: "REFUTES",
    2: "NOT ENOUGH INFO"
}

nli_model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=3,id2label=num2Label, label2id=label2Num)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def getWikiIdText(evidence, wiki_df):
    ids = []
    text = []
    for ann in evidence:
        for ann_data in ann:
            id = ann_data[2]
            if id != None and id not in ids:
                ids.append(id)
                _text = wiki_df.loc[id].values[0] if id in wiki_df.index else ""
                text.append(_text)
    return ids, text

class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, idx):
        claim = self.data.iloc[idx]["claim"]
        evidence = self.data.iloc[idx]["evidence"]
        ids, text = getWikiIdText(evidence, wiki_df)
        hyp = " [SEP] ".join(text)
        fin  = claim + " [SEP] " + hyp
        tokens = tokenizer(fin, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#         item = {key: torch.tensor(val[0]) for key, val in item.items()}
        correct = label2Num[self.data.iloc[idx]["label"]]
#         labels = [0,0,0]
#         labels[correct] = 1
#         labels = torch.FloatTensor(labels)
        labels = torch.tensor(label2Num[self.data.iloc[idx]["label"]])
#         item["labels"] = labels
#         print("checking ", idx, claim, ids)
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': labels}
    
    def __len__(self):
        return len(self.data)
        

In [22]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(p):
#     print("computing")
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=1, report_to="none", save_total_limit = 2, evaluation_strategy ="steps", load_best_model_at_end=True, eval_steps = 1500, save_steps=1500 )
train, test = train_test_split(train_df, test_size=0.2)
train_dataset = Dataset(train)
eval_dataset = Dataset(test)

trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
import re
from numpy import dot
from numpy.linalg import norm


def cos(a,b):
    return dot(a, b)/(norm(a)*norm(b))


def get_sentences(text):
    s = text.split(r"\d+\t")
    s = [s[i] for i in range(1, len(s))]
    return s

def get_best_sentences(model, query, ids, lines, k=5):
    embs = []
    idxs = []
    sents = []
    nums = []
    emb_query = model.encode([query])[0]
    for i,text in enumerate(texts):
        s = get_sentences(text)
        embs.extend([cos(model.encode([_s])[0], emb_query) for _s in s])
        sents.extend(s)
        idxs.extend([ids[i] for j in s])
        nums.extend([i for i in range(len(s))])
    amax = np.array(embs).argsort()[-k:][::-1]
    retSents = [sents[i] for i in amax]
    retIds = [[idxs[a], nums[a]] for a in amax]
    return retSents, retIds
        

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

nli_model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)

premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was good."

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)







In [ ]:
with torch.no_grad():
    for i, r in final_df.iterrows():
        tok = tokenizer(r)
        output = model(**tok)
        preds = torch.nn.functional.softmax(output.logits, dim=1)